In [2]:
from conflux_web3 import Web3 as ConfluxWeb3
from web3 import Web3
from web3.middleware.signing import construct_sign_and_send_raw_middleware

import json, dotenv
from typing import Any, TypedDict, assert_type

config = dotenv.dotenv_values(".env.mainnet")


def get_sk() -> str:
    return assert_type(config["SECRET_KEY"], str)


def get_e_web3() -> Web3:
    return Web3(Web3.HTTPProvider(config["EVM_URL"]))


def get_c_web3() -> ConfluxWeb3:
    return ConfluxWeb3(ConfluxWeb3.HTTPProvider(config["CORE_URL"]))


class Metadata(TypedDict):
    abi: Any
    bytecode: Any


def get_metadata(name: str) -> Metadata:
    with open(f"build/contracts/{name}.json") as f:
        d = json.load(f)
        return {
            "abi": d["abi"],
            "bytecode": d["bytecode"]
        }

c_web3 = get_c_web3()
e_web3 = get_e_web3()
c_web3.cfx.default_account = c_web3.account.from_key(get_sk())

print(f"deploy admin: {c_web3.cfx.default_account}")

e_account = e_web3.eth.account.from_key(get_sk())
e_web3.middleware_onion.add(construct_sign_and_send_raw_middleware(e_account))
e_web3.eth.default_account = e_account.address

DualSpaceNFTCore = c_web3.cfx.contract(**get_metadata("DualSpaceNFTCore"))
DualSpaceNFTEvm = e_web3.eth.contract(**get_metadata("DualSpaceNFTEvm"))
DeploymentProxyCore = c_web3.cfx.contract(**get_metadata("DeploymentProxy"))
DeploymentProxyEvm = e_web3.eth.contract(**get_metadata("DeploymentProxy"))
espace_chain_id = e_web3.eth.chain_id

core_proxy = DualSpaceNFTCore(config["CORE_PROXY"])
evm_proxy = DualSpaceNFTEvm(config["EVM_PROXY"])

deploy admin: cfx:aanthezva9vuyrpsmb15sf9xt97f8vduky1k1vwka5


In [3]:
print(f"deploying new impl...")
core_impl_addr = DualSpaceNFTCore.constructor().transact().executed()["contractCreated"]
print(f"core contract impl deployed at {core_impl_addr}")

evm_impl_addr = e_web3.eth.wait_for_transaction_receipt(DualSpaceNFTEvm.constructor().transact({
    "gasPrice": e_web3.eth.gas_price * 2,
}))["contractAddress"]
print(f"evm contract impl deployed at {evm_impl_addr}")


deploying new impl...
core contract impl deployed at cfx:acg3r2bxh15nw1386f02avvbv7a3tkm1328a007nf2
evm contract impl deployed at 0xa56c9DACCCaF10d323Cf56046c766913d5d09EA8


In [4]:
core_proxy.functions.upgradeTo(core_impl_addr).transact().executed()
print(f"core contract upgrade to {core_impl_addr}")

core_proxy.functions.upgradeEvmContractTo(evm_impl_addr).transact().executed()
print(f"evm contract upgrade to {evm_impl_addr}")

core contract upgrade to cfx:acg3r2bxh15nw1386f02avvbv7a3tkm1328a007nf2
evm contract upgrade to 0xa56c9DACCCaF10d323Cf56046c766913d5d09EA8
